In [5]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time

# set up Chrome driver
options = webdriver.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled')
driver = webdriver.Chrome(options=options)

# get Premier League stats page
driver.get("https://fbref.com/en/comps/9/Premier-League-Stats")
data = driver.page_source

# parse HTML with BeautifulSoup
soup = BeautifulSoup(data, 'html.parser')

# get all team URLs from standings table
standings_table = soup.select('table.stats_table')
if len(standings_table) > 0:
    links = [l.get("href") for l in standings_table[0].find_all('a')]
    links = [l for l in links if '/squads/' in l]
else:
    print("No standings table found.")

# create list to store match data for all teams
all_matches = []

# loop over all years from 2023 to 2020
for year in range(2023, 2019, -1):

    # loop over all team URLs
    for team_url in [f"https://fbref.com{l}" for l in links]:
        
        # get team name from URL
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-"," ")

        # get matches table for team
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

        # get shooting table for team
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()

        # merge matches and shooting tables and add season and team name columns
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]])
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        team_data["Season"] = year
        team_data["Team"] = team_name
        
        # append team data to list
        all_matches.append(team_data)
        
        # wait for 1 second to avoid overloading the server
        time.sleep(1)

# combine data for all teams into a single DataFrame
match_df = pd.concat(all_matches)

# convert column names to lowercase
match_df.columns = [c.lower() for c in match_df.columns]

# save data to CSV file
match_df.to_csv("matches2.csv")

# calculate and print average number of shots on target per game for each team in each season
for year in range(2023, 2019, -1):
    season_df = match_df[match_df["season"] == year]
    print(f"\nSeason {year}")
    for team in season_df["team"].unique():
        team_df = season_df[season_df["team"] == team]
        avg_sot_per_game = round(team_df["sot"].mean(), 2)
        print(f"{team}: {avg_sot_per_game} shots on target per game")


ValueError: No tables found

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# set variables
team_url = "https://www.premierleague.com/clubs/4/Chelsea/stats?co=1&se=418"
table_strings = ["Match Report", "Match Summary"]

# get matches table for team
data = requests.get(team_url)
print(data.text)  # print HTML code for debugging
for string in table_strings:
    try:
        matches = pd.read_html(data.text, match=string)[0]
        break
    except ValueError:
        continue
else:
    raise ValueError("No matching table found")

# get shooting table for team
soup = BeautifulSoup(data.text)
shooting_table = soup.find("div", {"class": "normalStatTables"})

<!DOCTYPE html>
<html lang="en">
<head>
    <meta name="twitter:title" content="Chelsea FC Statistics | Premier League"/>
<meta name="keywords" content="Premier League, Football, Soccer, Official"/>
<meta property="og:type" content="website"/>
<meta name="description" content="View the latest comprehensive Chelsea FC match stats, along with a season by season archive, on the official website of the Premier League."/>
<meta name="twitter:description" content="View the latest comprehensive Chelsea FC match stats, along with a season by season archive, on the official website of the Premier League."/>
<meta name="twitter:image" content="https://s3.amazonaws.com/premierleague-static-files/premierleague/pl_icon.png"/>
<meta property="og:title" content="Chelsea FC Statistics | Premier League"/>
<meta property="og:image" content="https://s3.amazonaws.com/premierleague-static-files/premierleague/pl_icon.png"/>
<meta name="theme-color" content="#38003C"/>
<title>Chelsea FC Statistics | Premier 

ValueError: No matching table found